In [1]:
%run imports.py

# Download food nutrient data

It is possible to download the nutrient data directly [1].
However, the latest release contains errors that have only been updated in the web interface.

[1] https://www.ars.usda.gov/northeast-area/beltsville-md/beltsville-human-nutrition-research-center/nutrient-data-laboratory/docs/usda-national-nutrient-database-for-standard-reference/

In [2]:
# Get a list of nutrient ids.
soup = bs4.BeautifulSoup(requests.get('https://ndb.nal.usda.gov/ndb/nutrients/index').text, 'html5lib')
nutrient_ids = [option.get('value') for option in soup.select('select#nutrient3 > option') if option.get('value')]

In [3]:
# For each nutrient, download a DataFrame of foods with that nutrient.
foods = {} if 'foods' not in locals() else foods
for nutrient_id in tqdm_notebook(nutrient_ids, unit='nutrient'):
    if nutrient_id in foods:
        continue
    url = 'https://ndb.nal.usda.gov/ndb/nutrients/report/nutrientsfrm'
    params = {
        'max': 25,
        'offset': 0,
        'totCount': 0,
        'nutrient1': nutrient_id,
        'nutrient2': '',
        'nutrient3': '',
        'subset': 1,  # 0=All foods, 1=Abridged (most common 1000 foods)
        'sort': 'f',
        'measureby': 'm'
    }
    soup = bs4.BeautifulSoup(requests.get(url, params).text, 'html5lib')
    url_csv = 'https://ndb.nal.usda.gov'
    url_csv += [a.get('href') for a in soup.select('a[title="Download in spreadsheet format"]')][0]
    src = requests.get(url_csv).text \
        .replace('Per Measure', '') \
        .replace('"""', '"') \
        .replace(',,\n', '\n') \
        .replace(',\n', '\n')
    df = pd.read_csv(
        io.StringIO(src),
        index_col='NDB_NO',
        skiprows=6)
    df = df[df.iloc[:, -1] > 0]
    foods[nutrient_id] = df

In [4]:
# Merge all nutrient DataFrames together.
nutrients_df = None
for nutrient_id, nutrient_df in tqdm_notebook(foods.items(), unit='nutrient', total=len(foods)):
    if nutrients_df is None:
        nutrients_df = nutrient_df
    else:
        nutrients_df = nutrients_df.combine_first(nutrient_df)

In [5]:
# Reorder the columns and store to disk.
cols_front = ['Description', 'Weight(g)', 'Measure']
cols_back = sorted([col for col in nutrients_df.columns if col not in cols_front])
nutrients_df = nutrients_df[cols_front + cols_back].sort_index()
nutrients_df.to_msgpack('nutrients.msg', compress='zlib')

In [6]:
nutrients_df[[col for col in nutrients_df.columns if ':' not in col]].head(20)

,Description,Weight(g),Measure,Alanine(g),"Alcohol, ethyl(g)",Arginine(g),Ash(g),Aspartic acid(g),Beta-sitosterol(mg),Betaine(mg),Caffeine(mg),"Calcium, Ca(mg)",Campesterol(mg),"Carbohydrate, by difference(g)","Carotene, alpha(µg)","Carotene, beta(µg)",Cholesterol(mg),"Choline, total(mg)","Copper, Cu(mg)","Cryptoxanthin, beta(µg)",Cystine(g),Dihydrophylloquinone(µg),Energy(kJ),Energy(kcal),"Fatty acids, total monounsaturated(g)","Fatty acids, total polyunsaturated(g)","Fatty acids, total saturated(g)","Fatty acids, total trans(g)","Fatty acids, total trans-monoenoic(g)","Fatty acids, total trans-polyenoic(g)","Fiber, total dietary(g)","Fluoride, F(µg)","Folate, DFE(µg)","Folate, food(µg)","Folate, total(µg)",Folic acid(µg),Fructose(g),Galactose(g),Glucose (dextrose)(g),Glutamic acid(g),Glycine(g),Histidine(g),Hydroxyproline(g),"Iron, Fe(mg)",Isoleucine(g),Lactose(g),Leucine(g),Lutein + zeaxanthin(µg),Lycopene(µg),Lysine(g),"Magnesium, Mg(mg)",Maltose(g),"Manganese, Mn(mg)",Menaquinone-4(µg),Methionine(g),Niacin(mg),Pantothenic acid(mg),Phenylalanine(g),"Phosphorus, P(mg)",Phytosterols(mg),"Potassium, K(mg)",Proline(g),Protein(g),Retinol(µg),Riboflavin(mg),"Selenium, Se(µg)",Serine(g),"Sodium, Na(mg)",Starch(g),Stigmasterol(mg),Sucrose(g),"Sugars, total(g)",Theobromine(mg),Thiamin(mg),Threonine(g),"Tocopherol, beta(mg)","Tocopherol, delta(mg)","Tocopherol, gamma(mg)","Tocotrienol, alpha(mg)","Tocotrienol, beta(mg)","Tocotrienol, delta(mg)","Tocotrienol, gamma(mg)",Total lipid (fat)(g),Tryptophan(g),Tyrosine(g),Valine(g),"Vitamin A, IU(IU)","Vitamin A, RAE(µg)",Vitamin B-12(µg),"Vitamin B-12, added(µg)",Vitamin B-6(mg),"Vitamin C, total ascorbic acid(mg)",Vitamin D (D2 + D3)(µg),Vitamin D(IU),Vitamin D2 (ergocalciferol)(µg),Vitamin D3 (cholecalciferol)(µg),Vitamin E (alpha-tocopherol)(mg),"Vitamin E, added(mg)",Vitamin K (phylloquinone)(µg),Water(g),"Zinc, Zn(mg)"
NDB_NO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,"Butter, salted",5.00,"1.0 pat (1"" sq, 1/3"" high)",0.001,NaN,0.002,0.11,0.003,NaN,NaN,NaN,1.0,NaN,NaN,NaN,8.0,11.0,0.9,NaN,NaN,NaN,NaN,150.0,36.0,1.051,0.152,2.568,0.164,0.149,0.015,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,0.001,0.001,NaN,NaN,0.003,NaN,0.004,NaN,NaN,0.003,NaN,NaN,NaN,NaN,0.001,0.002,0.006,0.002,1.0,NaN,1.0,0.004,0.04,34.0,0.002,0.1,0.002,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.06,0.001,0.002,0.003,125.0,34.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12,NaN,0.3,0.79,NaN
1004,"Cheese, blue",28.35,1.0 oz,0.183,NaN,0.202,1.45,0.407,NaN,NaN,NaN,150.0,NaN,0.66,NaN,21.0,21.0,4.4,0.011,NaN,0.030,NaN,419.0,100.0,2.205,0.227,5.293,NaN,NaN,NaN,NaN,NaN,10.0,10.0,10.0,NaN,NaN,NaN,NaN,1.468,0.115,0.215,NaN,0.09,0.319,NaN,0.544,NaN,NaN,0.525,7.0,NaN,0.003,NaN,0.166,0.288,0.490,0.308,110.0,NaN,73.0,0.595,6.07,54.0,0.108,4.1,0.318,325.0,NaN,NaN,NaN,0.14,NaN,0.008,0.223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.15,0.088,0.367,0.441,204.0,56.0,0.35,NaN,0.047,NaN,0.1,6.0,NaN,0.1,0.07,NaN,0.7,12.02,0.75
1007,"Cheese, camembert",28.35,1.0 oz,0.232,NaN,0.199,1.04,0.365,NaN,NaN,NaN,110.0,NaN,0.13,NaN,3.0,20.0,4.4,0.006,NaN,0.031,NaN,355.0,85.0,1.991,0.205,4.326,NaN,NaN,NaN,NaN,NaN,18.0,18.0,18.0,NaN,NaN,NaN,NaN,1.187,0.107,0.194,NaN,0.09,0.274,NaN,0.522,NaN,NaN,0.501,6.0,NaN,0.011,NaN,0.160,0.179,0.387,0.313,98.0,NaN,53.0,0.665,5.61,68.0,0.138,4.1,0.316,239.0,NaN,NaN,NaN,0.13,NaN,0.008,0.203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.88,0.087,0.325,0.363,232.0,68.0,0.37,NaN,0.064,NaN,0.1,5.0,NaN,0.1,0.06,NaN,0.6,14.69,0.67
1009,"Cheese, cheddar",132.00,"1.0 cup, diced",0.991,NaN,0.722,4.90,2.289,NaN,0.9,NaN,937.0,NaN,4.08,NaN,112.0,131.0,21.8,0.040,NaN,0.162,NaN,2229.0,533.0,12.205,1.876,24.904,1.210,1.210,NaN,NaN,46.1,36.0,36.0,36.0,NaN,NaN,0.13,0.34,6.250,0.722,0.722,NaN,0.18,1.592,0.16,2.559,NaN,NaN,1.353,36.0,NaN,0.036,11.4,0.722,0.078,0.541,1.418,601.0,NaN,100.0,3.296,30.19,436.0,0.565,37.6,1.030,862.0,NaN,NaN,NaN,0.63,NaN,0.038,1.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4

# Download Recommended Daily Allowances

In [7]:
def download_rda_table(url):
    # Download the RDA table at the given url.
    src = requests.get(url).text \
        .replace('Life Stage<br />Group', 'Group') \
        .replace('*', '')
    rda_df = pd.read_html(io.StringIO(src))[0]
    # Integrate group rows into the index.
    group = None
    for i, name in enumerate(rda_df.Group):
        if re.match('^[A-Z]', name):
            group = name
        else:
            rda_df.Group.iat[i] = f'{group}: {name}'
    # Clean up the data and dtypes.
    rda_df = rda_df.set_index('Group').dropna(axis=0, how='all')
    rda_df = rda_df.replace(['ND', 'NDc'], np.nan)
    rda_df = rda_df.replace('([0-9.]+)[^0-9]+', r'\1', regex=True)
    rda_df = rda_df.astype(np.float64)
    return rda_df

In [8]:
rda_elements_df = download_rda_table('https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t3/?report=objectonly')
rda_elements_df

,Calcium(mg/d),Chromium(μg/d),Copper(μg/d),Fluoride(mg/d),Iodine(μg/d),Iron(mg/d),Magnesium(mg/d),Manganese(mg/d),Molybdenum(μg/d),Phosphorus(mg/d),Selenium(μg/d),Zinc(mg/d),Potassium(g/d),Sodium(g/d),Chloride(g/d)
Group,,,,,,,,,,,,,,,
Infants: 0–6 mo,200.0,0.2,200.0,0.01,110.0,0.27,30.0,0.003,2.0,100.0,15.0,2.0,0.4,0.12,0.18
Infants: 6–12 mo,260.0,5.5,220.0,0.50,130.0,11.00,75.0,0.600,3.0,275.0,20.0,3.0,0.7,0.37,0.57
Children: 1–3 y,700.0,11.0,340.0,0.70,90.0,7.00,80.0,1.200,17.0,460.0,20.0,3.0,3.0,1.00,1.50
Children: 4–8 y,1000.0,15.0,440.0,1.00,90.0,10.00,130.0,1.500,22.0,500.0,30.0,5.0,3.8,1.20,1.90
Males: 9–13 y,1300.0,25.0,700.0,2.00,120.0,8.00,240.0,1.900,34.0,1250.0,40.0,8.0,4.5,1.50,2.30
Males: 14–18 y,1300.0,35.0,890.0,3.00,150.0,11.00,410.0,2.200,43.0,1250.0,55.0,11.0,4.7,1.50,2.30
Males: 19–30 y,1000.0,35.0,900.0,4.00,150.0,8.00,400.0,2.300,45.0,700.0,55.0,11.0,4.7,1.50,2.30
Males: 31–50 y,1000.0,35.0,900.0,4.00,150.0,8.00,420.0,2.300,45.0,700.0,55.0,11.0,4.7,1.50,2.30
Males: 51–70 y,1000.0,30.0,900.0,4.00,150.0,8.00,420.0,2.300,45.0,700.0,55.0,11.0,4.7,1.30,2.00


In [9]:
rda_vitamins_df = download_rda_table('https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t2/?report=objectonly')
rda_vitamins_df

,Vitamin A(μg/d)a,Vitamin C(mg/d),"Vitamin D(μg/d)b,c",Vitamin E(mg/d)d,Vitamin K(μg/d),Thiamin(mg/d),Riboflavin(mg/d),Niacin(mg/d)e,Vitamin B6(mg/d),Folate(μg/d)f,Vitamin B12(μg/d),Pantothenic Acid(mg/d),Biotin(μg/d),Choline(mg/d)g
Group,,,,,,,,,,,,,,
Infants: 0–6 mo,400.0,40.0,10.0,4.0,2.0,0.2,0.3,2.0,0.1,65.0,4.0,1.7,5.0,125.0
Infants: 6–12 mo,500.0,50.0,10.0,5.0,2.5,0.3,0.4,4.0,0.3,80.0,5.0,1.8,6.0,150.0
Children: 1–3 y,300.0,15.0,15.0,6.0,30.0,0.5,0.5,6.0,0.5,150.0,9.0,2.0,8.0,200.0
Children: 4–8 y,400.0,25.0,15.0,7.0,55.0,0.6,0.6,8.0,0.6,200.0,12.0,3.0,12.0,250.0
Males: 9–13 y,600.0,45.0,15.0,11.0,60.0,0.9,0.9,12.0,1.0,300.0,18.0,4.0,20.0,375.0
Males: 14–18 y,900.0,75.0,15.0,15.0,75.0,1.2,1.3,16.0,1.3,400.0,24.0,5.0,25.0,550.0
Males: 19–30 y,900.0,90.0,15.0,15.0,120.0,1.2,1.3,16.0,1.3,400.0,24.0,5.0,30.0,550.0
Males: 31–50 y,900.0,90.0,15.0,15.0,120.0,1.2,1.3,16.0,1.3,400.0,24.0,5.0,30.0,550.0
Males: 51–70 y,900.0,90.0,15.0,15.0,120.0,1.2,1.3,16.0,1.7,400.0,2.4,5.0,30.0,550.0


In [10]:
rda_macro_df = download_rda_table('https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t4/?report=objectonly')
rda_macro_df

,Total Watera(L/d),Carbohydrate(g/d),Total Fiber(g/d),Fat(g/d),Linoleic Acid(g/d),α-Linolenic Acid(g/d),Proteinb(g/d)
Group,,,,,,,
Infants: 0–6 mo,0.7,60.0,NaN,31.0,4.4,0.5,9.1
Infants: 6–12 mo,0.8,95.0,NaN,30.0,4.6,0.5,11.0
Children: 1–3 y,1.3,130.0,19.0,NaN,7.0,0.7,13.0
Children: 4–8 y,1.7,130.0,25.0,NaN,10.0,0.9,19.0
Males: 9–13 y,2.4,130.0,31.0,NaN,12.0,1.2,34.0
Males: 14–18 y,3.3,130.0,38.0,NaN,16.0,1.6,52.0
Males: 19–30 y,3.7,130.0,38.0,NaN,17.0,1.6,56.0
Males: 31–50 y,3.7,130.0,38.0,NaN,17.0,1.6,56.0
Males: 51–70 y,3.7,130.0,30.0,NaN,14.0,1.6,56.0


In [12]:
# Store the RDA of the elements, vitamins and macronutrients to disk.
rda = {
    'elements': rda_elements_df,
    'vitamins': rda_vitamins_df,
    'macronutrients': rda_macro_df
}
rda['macronutrients'].reset_index(inplace=True)
rda['macronutrients'].loc[rda['macronutrients'].Group == 'Lactation: 14–18', 'Group'] += ' y'
rda['macronutrients'].set_index('Group', inplace=True)
rda = pd.concat(rda.values(), axis=1).sort_index()
pd.to_msgpack('rda.msg', rda, compress='zlib')